# How to convert tools to OpenAI Functions

This notebook goes over how to use LangChain tools as OpenAI functions.

In [1]:
%pip install -qU langchain-community langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.tools import MoveFileTool
from langchain_core.messages import HumanMessage
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_openai import ChatOpenAI

In [3]:
model = ChatOpenAI(model="gpt-3.5-turbo")

In [4]:
tools = [MoveFileTool()]
functions = [convert_to_openai_function(t) for t in tools]

In [5]:
functions[0]

{'name': 'move_file',
 'description': 'Move or rename a file from one location to another',
 'parameters': {'properties': {'source_path': {'description': 'Path of the file to move',
    'type': 'string'},
   'destination_path': {'description': 'New path for the moved file',
    'type': 'string'}},
  'required': ['source_path', 'destination_path'],
  'type': 'object'}}

In [6]:
message = model.invoke(
    [HumanMessage(content="move file foo to bar")], functions=functions
)

In [7]:
message

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"source_path":"foo","destination_path":"bar"}', 'name': 'move_file'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 77, 'total_tokens': 97}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-7a79c849-aedc-47ed-8f0a-3d1791ee4a8c-0', usage_metadata={'input_tokens': 77, 'output_tokens': 20, 'total_tokens': 97})

In [8]:
message.additional_kwargs["function_call"]

{'arguments': '{"source_path":"foo","destination_path":"bar"}',
 'name': 'move_file'}

With OpenAI chat models we can also automatically bind and convert function-like objects with `bind_functions`

In [9]:
model_with_functions = model.bind_functions(tools)
model_with_functions.invoke([HumanMessage(content="move file foo to bar")])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"source_path":"foo","destination_path":"bar"}', 'name': 'move_file'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 77, 'total_tokens': 97}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-754057d0-7c9c-4db0-92fd-27ca8d503217-0', usage_metadata={'input_tokens': 77, 'output_tokens': 20, 'total_tokens': 97})

Or we can use the update OpenAI API that uses `tools` and `tool_choice` instead of `functions` and `function_call` by using `ChatOpenAI.bind_tools`:

In [10]:
model_with_tools = model.bind_tools(tools)
model_with_tools.invoke([HumanMessage(content="move file foo to bar")])

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_CaL8VjekqJ7gWLk3xREvRspm', 'function': {'arguments': '{"source_path":"foo","destination_path":"bar"}', 'name': 'move_file'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 77, 'total_tokens': 97}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6aaa7850-0a12-4879-bcce-8bc373b0c5c9-0', tool_calls=[{'name': 'move_file', 'args': {'source_path': 'foo', 'destination_path': 'bar'}, 'id': 'call_CaL8VjekqJ7gWLk3xREvRspm', 'type': 'tool_call'}], usage_metadata={'input_tokens': 77, 'output_tokens': 20, 'total_tokens': 97})